In [1]:
#imports
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import sys
#sys.path.append("../BerkeleySETI/Messy-Notebooks")
sys.path.append("../FinalClassifier/data")
import os

import data
from data import loaders
from data import phasefold as pf
import lightkurve as lk
import warnings
import glob
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
#set up
warnings.filterwarnings("ignore", category=RuntimeWarning) #fset = pd.read_csv('sect6data.csv')
data_dir = "/mnt/disks/lcs/"
data.mount_drive(data_dir)
data_dir = "/mnt/disks/lcs/tess-goddard-lcs/"

Disk mounted


mount: /mnt/disks/lcs: /dev/sdc already mounted on /mnt/disks/lcs.


___
We want to get the features that we will use for our classifier
<br><b> *Note: Be sure we have the correct 62 features, with mstat and astat

In [3]:
#import sector 6 features
!gcsfuse --implicit-dirs tess-goddard-lcs /home/jupyter/mountpoint
datafile = "/home/jupyter/mountpoint/dataproducts/db.v2.h5"
scores6 = pd.read_hdf(datafile, f'S6/scores')
features6 = pd.read_hdf(datafile, f'S6/features')

2023/01/23 20:43:25.838555 Start gcsfuse/0.39.2 (Go version go1.17.5) for app "" using mount point: /home/jupyter/mountpoint
2023/01/23 20:43:25.846615 Opening GCS connection...
2023/01/23 20:43:25.980441 Mounting file system "tess-goddard-lcs"...
daemonize.Run: readFromProcess: sub-process: mountWithArgs: mountWithConn: Mount: mount: running /usr/bin/fusermount: exit status 1


In [7]:
features6

____
You don't need to run the below section, as feats.csv has already been provided as training data. This is just for your own reference if you're wondering where it came from

In [4]:
#list of ebs that we will use for our training data
txt_file = open("data/ebs_s6.txt", "r")
file_content = txt_file.read()
content_list = file_content.split("\n")
txt_file.close()
eb = [eval(i) for i in content_list]

#list of eb tics
eb

[235231072,
 301185875,
 301185877,
 234169663,
 89016404,
 80836949,
 140336965,
 156501953,
 282503011,
 715766691,
 301185891,
 436619704,
 39354700,
 220140809,
 156241056,
 285000038,
 285000042,
 234065348,
 265899106,
 39838127,
 234878308,
 43144900,
 206794153,
 265715239,
 207004915,
 235548197,
 247681431,
 36288537,
 119461266,
 234169642,
 237590010,
 39667795,
 235552142,
 79635172,
 369673753,
 280212098,
 266193357,
 143215473,
 234926584,
 445955531,
 712487688,
 279900791,
 387690877,
 42991740,
 438545375,
 92009366,
 159086692,
 247556517,
 284199394,
 231161386,
 445955532,
 445955536,
 206840164,
 206840192,
 234158088,
 308387164,
 60242904,
 714514062,
 139476794,
 139476806,
 308169172,
 159057212,
 309908733,
 349480507,
 42885762,
 206840156,
 206850953,
 234146623,
 308169197,
 60242900,
 220149256,
 231959604,
 42991745,
 140209204,
 237647557,
 264594192,
 264594193,
 264594194,
 281157204,
 138904948,
 280304732,
 437918915,
 97568437,
 247681430,
 369681

In [5]:
f6eb = features6.loc[eb]

In [10]:
#f6eb - corresponding 62 features for each eb
f6eb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
235231072,0.026648,0.991050,0.000710,0.026889,94.0,0.0,0.0,0.0,0.000851,0.024911,...,0.881838,0.009123,0.013710,0.018850,0.026099,0.428787,0.108102,0.042574,1.569,0.000000
301185875,0.081945,0.971039,0.006715,0.084389,97.0,0.0,6.0,6.0,0.001877,0.500361,...,0.864516,0.004411,0.009037,0.013953,0.030553,0.436112,0.363679,0.042226,1.561,0.838213
301185877,0.081945,0.971039,0.006715,0.084389,97.0,0.0,6.0,6.0,0.001877,0.500361,...,0.864516,0.004411,0.009037,0.013953,0.030553,0.436112,0.363679,0.042226,1.561,0.838212
234169663,0.044620,0.984330,0.001991,0.045330,95.0,0.0,1.0,1.0,0.001174,0.165540,...,0.864516,0.006410,0.011026,0.016200,0.026155,0.478317,0.206123,0.024256,1.559,0.673318
89016404,0.086715,0.966162,0.007520,0.089752,112.0,0.0,0.0,0.0,0.002893,0.599635,...,0.835498,0.007718,0.014056,0.023216,0.110242,0.612072,0.357053,0.079822,1.558,0.848091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271643533,0.159520,0.949524,0.025447,0.168000,387.0,0.0,1.0,1.0,0.110968,5.774229,...,0.176963,0.249553,0.441912,0.638240,0.819257,0.941121,0.968383,0.385284,0.623,0.934289
436423408,0.214966,0.928026,0.046210,0.231638,361.0,0.0,0.0,0.0,0.147812,5.761445,...,0.211039,0.227504,0.422346,0.601965,0.813311,0.927010,0.507398,0.340901,0.622,0.901662
33865219,0.178479,0.943878,0.031855,0.189092,339.0,0.0,1.0,1.0,0.118789,6.655849,...,0.214831,0.226883,0.412879,0.600523,0.794949,0.928034,1.000000,0.410119,0.621,0.899500
290039579,0.184797,0.941332,0.034150,0.196315,374.0,0.0,1.0,1.0,0.128155,5.482608,...,0.186848,0.235463,0.431017,0.645656,0.826721,0.941337,0.910452,0.456640,0.596,0.926016


In [6]:
#list of ebs from villanova (should be exhaustive in tmcl), which we will use to find our noneb dataset
tmclebs = pd.read_csv('data/allsectorstmclebs.csv')

ref = pd.read_csv(data_dir+f"sector6lookup.csv")
tmcl = ["2_min_cadence" in fn for fn in ref.Filename]
reftmcl = ref[tmcl]
z = ~reftmcl.TIC_ID.isin(tmclebs.TIC_ID.values)
f6nonebs = reftmcl[z].sample(frac=1)

In [7]:
#features for our noneb
f6noneb = features6.loc[f6nonebs.TIC_ID.values]
f6noneb

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
141752883,0.032362,0.999058,1.047304e-03,0.032393,1.0,0.0,1.0,1.0,0.000639,0.021783,...,0.306806,0.171059,0.303348,0.449553,0.596100,0.794872,1.000000,0.679216,0.159,0.117348
20209536,0.032465,0.998603,1.053968e-03,0.032510,1.0,0.0,1.0,1.0,0.003768,0.023687,...,0.195607,0.240219,0.432475,0.617447,0.754818,0.898587,0.995625,0.612232,0.174,0.542287
238197946,0.033006,0.998895,1.089396e-03,0.033042,1.0,0.0,1.0,1.0,0.000192,0.017597,...,0.404580,0.136334,0.250498,0.365908,0.557848,0.745629,1.000000,1.132634,0.165,0.036213
42457828,0.000386,0.999974,1.489859e-07,0.000386,239.0,0.0,6.0,6.0,0.000228,0.007756,...,0.452688,0.150431,0.286115,0.414842,0.579236,0.783798,0.002507,0.440111,0.490,0.092848
238931356,0.032777,0.999050,1.074316e-03,0.032808,1.0,0.0,1.0,1.0,0.001410,0.069156,...,0.383459,0.146753,0.267624,0.411022,0.552759,0.757665,0.998427,0.786770,0.150,0.291584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80668606,0.001833,1.000093,3.360324e-06,0.001833,230.0,4.0,0.0,4.0,0.001083,0.073460,...,0.345238,0.164935,0.296555,0.423565,0.578712,0.803717,0.019683,2.619319,-0.439,0.064997
177236998,0.032438,0.998957,1.052246e-03,0.032472,2.0,0.0,1.0,1.0,0.001138,0.097011,...,0.411518,0.149418,0.253327,0.388944,0.549718,0.762282,1.000000,1.488727,0.196,0.110658
427377568,0.000378,0.999972,1.427930e-07,0.000378,258.0,0.0,4.0,4.0,0.000233,0.013577,...,0.413136,0.148675,0.260505,0.385639,0.535393,0.716157,0.001923,0.541293,0.335,0.252137
443031162,0.032913,0.998936,1.083282e-03,0.032948,4.0,0.0,1.0,1.0,0.003642,0.280861,...,0.382600,0.163423,0.276762,0.407092,0.546809,0.759169,1.000000,0.732346,0.183,0.052376


In [10]:
#new dataset (3) = f6eb + f6noneb
frames = [f6noneb, f6eb]
feats = pd.concat(frames)
feats.to_csv('data/feats.csv')

___
ML portion here

In [8]:
#run this line to read in feats.csv
feats = pd.read_csv('data/feats.csv', index_col = 0)

In [9]:
train, test = train_test_split(np.arange(len(feats)), train_size=0.8)
labels = []

for i in range(len(f6noneb)):
    labels.append('NonEB')
for i in range(len(f6noneb),len(f6eb)+len(f6noneb)):
    labels.append('EB')
    
X_trainfull = feats.iloc[train]
X_testfull = feats.iloc[test]

y_train = []
y_test = []
for i in train:
    y_train.append(labels[i])
for i in test:
    y_test.append(labels[i])
    
y_train = pd.Series(data = y_train, index = train)
y_test = pd.Series(data = y_test, index = test)

In [10]:
newy_train = []
for i in (y_train.index.values):
    if (y_train[i]=='NonEB'):
        newy_train.append(0)
    else:
        newy_train.append(1)
y_train = pd.Series(data = newy_train, index = train)

newy_test = []
for i in (y_test.index.values):
    if (y_test[i]=='NonEB'):
        newy_test.append(0)
    else:
        newy_test.append(1)
y_test = pd.Series(data = newy_test, index = test)

In [11]:
X_train = X_trainfull
X_test = X_testfull

In [16]:
X_train

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,mstat,astat
32034265,0.033009,0.998936,0.001090,0.033044,1.0,0.0,1.0,1.0,0.000280,0.015626,...,0.432933,0.143311,0.264984,0.393812,0.532299,0.766463,1.000000,1.273311,0.164,0.235399
238949526,0.032758,0.998940,0.001073,0.032793,1.0,0.0,1.0,1.0,0.000201,0.012172,...,0.371643,0.150027,0.270864,0.412247,0.558991,0.769680,1.000000,0.641473,0.162,0.232014
67197292,0.032294,0.998965,0.001043,0.032328,1.0,0.0,1.0,1.0,0.000257,0.021495,...,0.376827,0.147975,0.266305,0.398647,0.565261,0.770930,1.000000,0.814851,0.160,0.076123
255683786,0.032777,0.998858,0.001074,0.032814,1.0,0.0,1.0,1.0,0.000834,0.012149,...,0.335124,0.154607,0.302515,0.408824,0.549936,0.751167,1.000000,0.571830,0.165,0.631518
24948729,0.032404,0.998913,0.001050,0.032439,1.0,0.0,1.0,1.0,0.001013,0.086636,...,0.402516,0.132812,0.246446,0.360943,0.520940,0.751524,1.000000,0.663836,0.167,0.031053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278899461,0.032479,0.998945,0.001055,0.032514,1.0,0.0,1.0,1.0,0.000153,0.007374,...,0.366420,0.155859,0.269150,0.423038,0.582543,0.804810,1.000000,0.724643,0.161,0.406353
167125688,0.032348,0.998960,0.001046,0.032382,1.0,0.0,1.0,1.0,0.000335,0.025409,...,0.518325,0.141360,0.249840,0.374989,0.520015,0.765963,1.000000,0.806458,0.164,0.173805
219183103,0.032757,0.998931,0.001073,0.032792,1.0,0.0,1.0,1.0,0.000181,0.012155,...,0.392052,0.141220,0.259406,0.385886,0.538042,0.743628,1.000000,0.591531,0.163,0.099505
251236566,0.089483,0.959907,0.008007,0.093220,165.0,0.0,0.0,0.0,0.015844,0.653571,...,0.693548,0.040739,0.083116,0.206994,0.511864,0.830851,0.296436,0.169804,1.249,0.867780


In [12]:
RF = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators = 500, max_depth = 15,random_state=31415))
RF.fit(X_train,y_train)
y_predRF = RF.predict(X_test)
print(accuracy_score(y_test,y_predRF)) # Fraction of classifications that were correct
print(precision_score(y_test,y_predRF,pos_label=1))

0.9997580450036293
0.997867803837953


In [14]:
y_predprob = RF.predict_proba(X_test)

In [24]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predRF)

array([[3636,    1],
       [   0,  496]])

In [19]:
featswithlabels = feats
featswithlabels['Label'] = labels
featswithlabels = featswithlabels.sample(frac=1,random_state = 31415)
ylabel = featswithlabels['Label']
y = pd.Series(ylabel.values)
featswithlabels = featswithlabels.drop(columns=['Label'])
X = featswithlabels

In [20]:
models = [RF]
for i in models:
    scores = cross_val_score(i, X, y, cv=10)
    print(f'{i.named_steps}: {sum(scores)/10}')

{'standardscaler': StandardScaler(), 'randomforestclassifier': RandomForestClassifier(max_depth=15, n_estimators=500, random_state=31415)}: 0.9995160431451506


__________
This assumes that astats is already in the datafile, and can be imported along with the other features

In [16]:
#sector you want to classify
sector = 6

In [22]:
#create the directory for each sector
os.mkdir(f'Sector{sector}')

FileExistsError: [Errno 17] File exists: 'Sector1'

In [17]:
#change the variable toClassify to data that you want to classify
toClassify = pd.read_hdf(datafile, f'S{sector}/features')
refsect = pd.read_csv(data_dir+f"sector{sector}lookup.csv")

In [18]:
y_pred = RF.predict(toClassify)
y_predprob = RF.predict_proba(toClassify)

In [19]:
y_probzero = []
y_probone = []
for i in range(len(y_predprob)):
    y_probzero.append(y_predprob[i][0])
    y_probone.append(y_predprob[i][1])

In [20]:
predictions = pd.DataFrame(index = toClassify.index.values)
predictions['probzero'] = y_probzero
predictions['probone'] = y_probone
predictions['pred'] = y_pred
predictions = predictions.sort_values(by=['probone'], ascending = False)

In [48]:
predictions

,probzero,probone,pred
140608211,0.0,1.0,1
231791552,0.0,1.0,1
300740040,0.0,1.0,1
181052118,0.0,1.0,1
38811805,0.0,1.0,1
...,...,...,...
389505262,1.0,0.0,0
280048495,1.0,0.0,0
404808101,1.0,0.0,0
284196526,1.0,0.0,0


In [21]:
rslt_df = predictions[predictions['pred'] == 1]
rslt_df.to_csv(f'Sector{sector}/predSector{sector}.csv')
predictions.to_csv(f'Sector{sector}/sect{sector}predandprob.csv')

In [59]:
p = predictions[predictions['probone'] == 1]

,probzero,probone,pred
140608211,0.0,1.0,1
231791552,0.0,1.0,1
300740040,0.0,1.0,1
181052118,0.0,1.0,1
38811805,0.0,1.0,1
...,...,...,...
260950407,0.0,1.0,1
349641813,0.0,1.0,1
33734375,0.0,1.0,1
141122198,0.0,1.0,1


In [68]:
p = predictions[predictions['probone'] > 0.9]
p

,probzero,probone,pred
140608211,0.000,1.000,1
231791552,0.000,1.000,1
300740040,0.000,1.000,1
181052118,0.000,1.000,1
38811805,0.000,1.000,1
...,...,...,...
206498836,0.098,0.902,1
349683770,0.098,0.902,1
419588384,0.098,0.902,1
281630204,0.098,0.902,1


In [69]:
p = predictions[predictions['probone'] > 0.5]
p = p[p['probone'] < 0.9]
p

,probzero,probone,pred
270221859,0.102000,0.898000,1
126803454,0.102000,0.898000,1
206488769,0.102000,0.898000,1
231702259,0.102000,0.898000,1
1989908229,0.102000,0.898000,1
...,...,...,...
425064789,0.498000,0.502000,1
212255303,0.498000,0.502000,1
269883697,0.499000,0.501000,1
55274131,0.499000,0.501000,1


In [51]:
#rslt_df = pd.read_csv(f'predSector{sector}.csv', index_col = 0)
values = rslt_df.index.values
values = values.tolist()
with open(f"Sector{sector}/ticpredictionsSector{sector}.txt", "w") as output:
    for i in range(len(values)):
        output.write(str(values[i]))
        output.write("\n")

In [52]:
values

[140608211,
 231791552,
 300740040,
 181052118,
 38811805,
 303909865,
 126605150,
 270203709,
 204497617,
 307364443,
 117544914,
 89018905,
 349835374,
 316852879,
 33715938,
 316937318,
 126945917,
 306573146,
 370239669,
 126944778,
 273550448,
 39017668,
 220388567,
 290495367,
 327939046,
 177236393,
 2026930048,
 306665786,
 31798977,
 238891804,
 232087348,
 231093356,
 278666918,
 219170666,
 261265716,
 33734376,
 267091837,
 141866705,
 89043105,
 92259410,
 425935688,
 294671852,
 220491219,
 389021724,
 117544922,
 306900745,
 410354938,
 302751961,
 300014528,
 270174474,
 201256776,
 181886413,
 355545390,
 234524824,
 231714659,
 234285429,
 219335483,
 177164595,
 389020350,
 389020349,
 30266257,
 260503860,
 309781192,
 734588359,
 300013254,
 167750920,
 314887863,
 141268467,
 300657354,
 219159872,
 277773589,
 167651110,
 355446601,
 382627633,
 126945904,
 126945919,
 150389733,
 33862174,
 150390197,
 394700186,
 150361911,
 260541900,
 382627629,
 140761918,
 

---

In [4]:
sec1 = pd.read_csv('Sector1/sect1predandprob.csv', index_col=0)

In [8]:
eblist = sec1[sec1['probone'] > 0.5]
eblist

,probzero,probone,pred
140608211,0.000000,1.000000,1
231791552,0.000000,1.000000,1
300740040,0.000000,1.000000,1
181052118,0.000000,1.000000,1
38811805,0.000000,1.000000,1
...,...,...,...
425064789,0.498000,0.502000,1
212255303,0.498000,0.502000,1
269883697,0.499000,0.501000,1
55274131,0.499000,0.501000,1


In [9]:
values = sec1.index.values
ref = pd.read_csv(data_dir+f"sector1lookup.csv")

In [12]:
values

array([140608211, 231791552, 300740040, ..., 404808101, 284196526,
       255567371])

In [17]:
tics = eblist.index.values
len(tics)

4824

In [18]:
tics

array([140608211, 231791552, 300740040, ..., 269883697,  55274131,
       309860772])

In [61]:
ebvals = eblist[eblist['probone'] == 1].index.values
ebvals

array([ 140608211,  231791552,  300740040,  181052118,   38811805,
        303909865,  126605150,  270203709,  204497617,  307364443,
        117544914,   89018905,  349835374,  316852879,   33715938,
        316937318,  126945917,  306573146,  370239669,  126944778,
        273550448,   39017668,  220388567,  290495367,  327939046,
        177236393, 2026930048,  306665786,   31798977,  238891804,
        232087348,  231093356,  278666918,  219170666,  261265716,
         33734376,  267091837,  141866705,   89043105,   92259410,
        425935688,  294671852,  220491219,  389021724,  117544922,
        306900745,  410354938,  302751961,  300014528,  270174474,
        201256776,  181886413,  355545390,  234524824,  231714659,
        234285429,  219335483,  177164595,  389020350,  389020349,
         30266257,  260503860,  309781192,  734588359,  300013254,
        167750920,  314887863,  141268467,  300657354,  219159872,
        277773589,  167651110,  355446601,  382627633,  126945

In [62]:
refebs = ref.loc[ref['TIC_ID'].isin(ebvals)]
files = data_dir + refebs.Filename.values
filepath = files[0]

In [63]:
#100%ebs
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [64]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"Sector1/100%ebs.pdf") as pdf:
    plt.cla()
    for i in range(len(ebvals)):
        lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

In [79]:
#90%-100%ebs
ebvals = eblist[eblist['probone'] > 0.9]
ebvals = ebvals[ebvals['probone'] < 1].index.values
refebs = ref.loc[ref['TIC_ID'].isin(ebvals)]
files = data_dir + refebs.Filename.values
filepath = files[0]
len(ebvals)

1152

In [ ]:
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [80]:
import matplotlib.pyplot as plt

from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(f"Sector1/90-100%ebs.pdf") as pdf:
    plt.cla()
    for i in range(len(ebvals)):
        lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

____
classified as ebs (50%>) 
<br>
Finding and removing closest stars, then printing the remaining light curves into a pdf file to visually examine

In [21]:

stars = ref.TIC_ID.isin(tics)

In [22]:
#ref7335.csv - csv file for all ebs found
ref[stars].to_csv(f"Sector{sector}/refSect{sector}.csv")

In [23]:
closestarslist = []
for i in range(len(tics)):
    rastar = ref[stars].RA.values[i]
    decstar = ref[stars].Dec.values[i]
    dist = np.sqrt(((rastar-ref[stars].RA.values)*(np.cos(decstar)))**2+(decstar-ref[stars].Dec.values)**2)
    closestars = np.where((dist < 0.1) & (dist != 0))
    closestarslist.append(closestars)

In [24]:
closestarslist

[(array([], dtype=int64),),
 (array([ 350, 1513, 2081]),),
 (array([], dtype=int64),),
 (array([691]),),
 (array([1600]),),
 (array([ 359,  696, 1605]),),
 (array([355]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([ 731, 1664]),),
 (array([], dtype=int64),),
 (array([1679]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([1720, 1721, 1722]),),
 (array([], dtype=int64),),
 (array([1753]),),
 (array([ 375, 1755, 1788]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([ 793,  794, 4792]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([2023]),),
 (array([2043]),),
 (array([], dtype=int64),),
 (array([ 425, 2050]),),
 (array([], dtype=int64),),
 (array([877]),),
 (array([], dtype=int64),),
 (array([2069]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([455]),),
 (array([38]),),
 (array([37]),),
 (array([], dty

In [25]:
closestarstics = []
for i in range (len(closestarslist)):
    tics = []
    tics.append(ref[stars].TIC_ID.values[i])
    for j in range(len(closestarslist[i][0])):
        tics.append(ref[stars].TIC_ID.values[closestarslist[i][0][j]])
    closestarstics.append(tics)

In [26]:
closestarstics

[[269762258],
 [263074592, 263074589, 612155681, 263074593],
 [278706358],
 [261656371, 261656369],
 [152825521, 152825523],
 [139699256, 2026930048, 139699260, 139699248],
 [139721086, 139721085],
 [139841210],
 [214198381],
 [144462697],
 [389726435, 389726434, 389726433],
 [160036449],
 [229057817, 229056112],
 [204497617],
 [441406061],
 [152223725],
 [25226885, 25226877, 25226886, 25244926],
 [280052690],
 [358457398, 299835005],
 [141268467, 141268468, 141268471, 141268474],
 [33862174],
 [267263253],
 [79403459],
 [79225651, 79225639, 79225655, 1989169801],
 [99465976],
 [201176965],
 [389476160, 389476161],
 [261746657, 261748317],
 [262958558],
 [290547036, 290547040, 290547046],
 [263078276],
 [33715938, 33715940],
 [303909865],
 [271596290, 271596289],
 [220474209],
 [278683641],
 [201256771, 201256776],
 [92352621, 92352620],
 [92352620, 92352621],
 [441422220],
 [89398867],
 [147083089],
 [206462116, 2028782097, 206462118, 2028782096],
 [117544915, 117544922, 117544914],
 

In [32]:
#get lcs for ebs in the larger list thats not in smaller(same depth dip) eb list
refebs = ref.loc[ref['TIC_ID'].isin(eblist.index.values)]
files = data_dir + refebs.Filename.values
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [33]:
len(lcs)

4824

In [47]:
tic = []
avgflux = []
    
for i in range(len(lcs)):
    tic.append(refebs['TIC_ID'].values[i])
    avgflux.append(np.median(lcs[i].flux.value))

In [50]:
d = {'TIC': tic, 'avgflux': avgflux}
avg = pd.DataFrame(d)

In [51]:
avg

,TIC,avgflux
0,269762258,443.491322
1,263074592,30010.553216
2,278706358,8850.857966
3,261656371,9947.039924
4,152825521,8615.038484
...,...,...
4819,278861826,10458.540525
4820,309853072,14305.984937
4821,349572805,10619.731527
4822,234509218,12718.302203


In [52]:
highestfluxtics = []
for i in range(len(closestarstics)):
    a = []
    for j in closestarstics[i]:
        a.append(avg[avg['TIC'] == j].avgflux.values[0])
    highestfluxtics.append(closestarstics[i][a.index(max(a))])

In [53]:
highestfluxtics

[269762258,
 263074592,
 278706358,
 261656371,
 152825521,
 139699256,
 139721085,
 139841210,
 214198381,
 144462697,
 389726435,
 160036449,
 229057817,
 204497617,
 441406061,
 152223725,
 25226886,
 280052690,
 358457398,
 141268467,
 33862174,
 267263253,
 79403459,
 79225651,
 99465976,
 201176965,
 389476160,
 261746657,
 262958558,
 290547036,
 263078276,
 33715938,
 303909865,
 271596290,
 220474209,
 278683641,
 201256776,
 92352621,
 92352621,
 441422220,
 89398867,
 147083089,
 206462116,
 117544915,
 139148070,
 139207681,
 91369561,
 44623916,
 144194307,
 126944775,
 126945917,
 167692429,
 410354930,
 32200310,
 290131778,
 281498280,
 220521199,
 234491752,
 38586438,
 55652788,
 349832824,
 147131974,
 91448148,
 147201138,
 117515123,
 121420805,
 89018905,
 126602778,
 141606986,
 197766143,
 266996791,
 219308281,
 206505572,
 260161144,
 403153647,
 265591866,
 219338651,
 231714759,
 54001270,
 220388567,
 381857817,
 394006752,
 270204401,
 419744996,
 35044341

In [54]:
def checkIfDuplicates_2(listOfElems):
    ''' Check if given list contains any duplicates '''    
    setOfElems = set()
    for elem in listOfElems:
        if elem in setOfElems:
            return True
        else:
            setOfElems.add(elem)         
    return False

In [55]:
result = checkIfDuplicates_2(highestfluxtics)
if result:
    print('Yes, list contains duplicates')
else:
    print('No duplicates found in list')   

Yes, list contains duplicates


In [56]:
newlist = [] #eb list with no duplicates, cut down to ones with highest flux values in a radius of 0.1
duplist = []
for i in highestfluxtics:
    if i not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)

In [58]:
len(newlist)

2651

In [59]:
with open(f"Sector{sector}/newlistSect{sector}.txt", "w") as output:
    for i in range(len(newlist)):
        output.write(str(newlist[i]))
        output.write("\n")

In [61]:
refebs = ref.loc[ref['TIC_ID'].isin(newlist)]
files = data_dir + refebs.Filename.values
filepath = files[0]
refebs

,Filename,RA,Dec,TIC_ID,Sector,Camera,CCD,Magnitude
571,tesslcs_sector_1_104/2_min_cadence_targets/tes...,66.470863,-81.319731,269762258,1,3,3,13.024600
715,tesslcs_sector_1_104/2_min_cadence_targets/tes...,16.409233,-80.061304,263074592,1,3,3,9.358700
758,tesslcs_sector_1_104/2_min_cadence_targets/tes...,334.810708,-40.533673,278706358,1,1,2,10.389200
797,tesslcs_sector_1_104/2_min_cadence_targets/tes...,20.218095,-80.316844,261656371,1,3,3,10.566000
849,tesslcs_sector_1_104/2_min_cadence_targets/tes...,344.569628,-41.826179,152825521,1,2,3,9.915200
...,...,...,...,...,...,...,...,...
1190317,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,100.658603,-56.541875,278861826,1,4,4,9.869500
1190890,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,126.553872,-63.101620,309853072,1,4,3,9.678800
1190959,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,111.752572,-63.734975,349572805,1,4,3,9.833201
1191688,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,12.913047,-65.340492,234509218,1,3,1,9.795600


In [62]:
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [67]:

with PdfPages(f"Sector1/newlist.pdf") as pdf:
    plt.cla()
    for i in range(len(newlist)):
        lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

___
classified as ebs (90%> but not 100%) 
<br>
Finding and removing closest stars, then printing the remaining light curves into a pdf file to visually examine

In [69]:
eb90 = sec1[sec1['probone'] > 0.9]

In [76]:
tics90 = eb90.index.values
len(tics90)

1489

In [78]:
stars90 = ref.TIC_ID.isin(tics90)
ref[stars90]

,Filename,RA,Dec,TIC_ID,Sector,Camera,CCD,Magnitude
849,tesslcs_sector_1_104/2_min_cadence_targets/tes...,344.569628,-41.826179,152825521,1,2,3,9.91520
873,tesslcs_sector_1_104/2_min_cadence_targets/tes...,322.427402,-50.342173,139699256,1,2,4,7.14870
1213,tesslcs_sector_1_104/2_min_cadence_targets/tes...,325.456432,-50.169363,139841210,1,2,4,10.10110
1415,tesslcs_sector_1_104/2_min_cadence_targets/tes...,336.348309,-44.594226,144462697,1,2,3,11.83150
1656,tesslcs_sector_1_104/2_min_cadence_targets/tes...,336.752288,-37.188368,160036449,1,1,2,10.80220
...,...,...,...,...,...,...,...,...
1188378,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,327.590236,-54.399737,370239669,1,2,4,9.26240
1188651,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,332.331275,-18.988099,206498875,1,1,3,9.18637
1189699,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,87.541315,-56.806097,734511288,1,4,4,9.53750
1190890,tesslcs_sector_1_104/tesslcs_tmag_9_10/tesslc_...,126.553872,-63.101620,309853072,1,4,3,9.67880


In [74]:
closestarslist90 = []
for i in range(len(tics90)):
    rastar = ref[stars90].RA.values[i]
    decstar = ref[stars90].Dec.values[i]
    dist = np.sqrt(((rastar-ref[stars90].RA.values)*(np.cos(decstar)))**2+(decstar-ref[stars90].Dec.values)**2)
    closestars = np.where((dist < 0.1) & (dist != 0))
    closestarslist90.append(closestars)

In [75]:
closestarslist90

[(array([614]),),
 (array([205, 357, 616]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([626]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([640]),),
 (array([], dtype=int64),),
 (array([653]),),
 (array([212, 655]),),
 (array([], dtype=int64),),
 (array([ 391, 1469]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([240, 765]),),
 (array([425]),),
 (array([], dtype=int64),),
 (array([780]),),
 (array([256]),),
 (array([], dtype=int64),),
 (array([  81,  581, 1481]),),
 (array([455, 456]),),
 (array([387]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([850]),),
 (array([834, 835, 846, 852]),),
 (array([231, 464]),),
 (array([], dtype=int64),),
 (array([277]),),
 (array([478]),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([], dtype=int64),),
 (array([ 493, 1094]),),
 (array(

In [79]:
closestarstics90 = []
for i in range (len(closestarslist90)):
    tics = []
    tics.append(ref[stars90].TIC_ID.values[i])
    for j in range(len(closestarslist90[i][0])):
        tics.append(ref[stars90].TIC_ID.values[closestarslist90[i][0][j]])
    closestarstics90.append(tics)

In [80]:

refebs = ref.loc[ref['TIC_ID'].isin(eb90.index.values)]
files = data_dir + refebs.Filename.values
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [81]:
len(lcs)

1489

In [82]:
tic = []
avgflux = []
    
for i in range(len(lcs)):
    tic.append(refebs['TIC_ID'].values[i])
    avgflux.append(np.median(lcs[i].flux.value))

In [83]:
df = {'TIC': tic, 'avgflux': avgflux}
avg = pd.DataFrame(df)

In [84]:
highestfluxtics = []
for i in range(len(closestarstics90)):
    a = []
    for j in closestarstics90[i]:
        a.append(avg[avg['TIC'] == j].avgflux.values[0])
    highestfluxtics.append(closestarstics90[i][a.index(max(a))])

In [86]:
result = checkIfDuplicates_2(highestfluxtics)
if result:
    print('Yes, list contains duplicates')
else:
    print('No duplicates found in list')   

Yes, list contains duplicates


In [87]:
newlist = [] #eb list with no duplicates, cut down to ones with highest flux values in a radius of 0.1
duplist = []
for i in highestfluxtics:
    if i not in newlist:
        newlist.append(i)
    else:
        duplist.append(i)

In [88]:
lcc = loaders.LightCurveCollection(refebs.Sector.values,
                                   refebs.Camera.values,
                                   files,
                                   False
                            )
lcc.useCpus = 8
lcc.load_all_lcs()

lcs = lcc.lcs

In [ ]:
with PdfPages(f"Sector1/90-100%ebsmodified.pdf") as pdf:
    plt.cla()
    for i in range(len(newlist)):
        lcs[i].scatter()
        pdf.savefig()
        plt.close();
plt.close()

In [90]:
len(newlist)

998

___

In [5]:
sec1.loc[[247878713]]

KeyError: "None of [Int64Index([247878713], dtype='int64')] are in the [index]"

In [22]:
sec6 = pd.read_csv('Sector6/sect6predandprob.csv', index_col=0)

In [23]:
sec6.loc[[247878713]]

,probzero,probone,pred
247878713,0.772,0.228,0


In [24]:
sec6.loc[[282270730]]

,probzero,probone,pred
282270730,0.474,0.526,1


In [26]:
sec6.loc[[307380314]]

,probzero,probone,pred
307380314,0.01,0.99,1
